# DNA Challenge

## 📚 Цель проекта

Ваша задача — построить **векторные представления (эмбеддинги) ДНК-последовательностей**, и затем обучить классификаторы, чтобы достичь **максимально возможного качества на 5 биоинформатических задачах** из бенчмарка [Nucleotide Transformer Benchmark](https://huggingface.co/datasets/InstaDeepAI/nucleotide_transformer_downstream_tasks).

---

## 🧪 Датасеты и метрики

Вы будете работать с пятью подзадачами из бенчмарка. Для каждой задачи указана **основная метрика**, по которой оценивается качество модели:

---

## 🧬 Биологический смысл задач

| №  | Название задачи      | Метрика оценки | Биологическое значение |
|----|----------------------|----------------|-------------------------|
| 1  | **promoter_all**     | F1-score       | **Промотеры** — участки ДНК, с которых начинается транскрипция гена. Эти регуляторные элементы находятся перед началом гена и служат якорем для РНК-полимеразы. Задача важна для определения того, какие участки генома активно экспрессируются. |
| 2  | **enhancers**        | MCC            | **Энхансеры** — участки ДНК, усиливающие экспрессию генов. Они могут находиться далеко от самого гена и взаимодействовать с промотером через 3D-свёртку хроматина. Энхансеры играют ключевую роль в тканеспецифической экспрессии. Задача сложная из-за вариабельности энхансерных последовательностей. |
| 3  | **splice_sites_all** | Accuracy       | **Splice sites (сайты сплайсинга)** — границы между экзонами и интронами в пре-мРНК. Правильное определение этих сайтов важно для моделирования посттранскрипционных модификаций. Ошибки в сплайсинге часто приводят к заболеваниям. Задача — определить, где начинается и заканчивается сплайсинг. |
| 4  | **H3**               | MCC            | Метка **H3 (гистон H3, например, H3K4me3)** связана с **активными промотерами** и **регуляторными регионами**. Предсказание таких эпигенетических меток по ДНК-последовательности позволяет понять, какие регионы могут быть "включены" или "выключены" на уровне хроматина. |
| 5  | **H4**               | MCC            | **H4** — ещё один гистон, участвующий в **формировании нуклеосом** и упаковке ДНК. Его модификации также играют роль в регуляции транскрипции. Задача — определить участки ДНК, которые, вероятно, взаимодействуют с модифицированным H4, что даёт ключ к пониманию структуры хроматина. |

---

## 📈 Метрики качества

Ниже представлены формулы всех метрик, которые используются для оценки:

### 1. **F1-score (гармоническое среднее Precision и Recall)**

F1-score особенно важна в задачах со **сильным дисбалансом классов**.

$$
\text{F1} = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}, \quad \text{где:}
$$

$$
\text{Precision} = \frac{TP}{TP + FP}, \quad \text{Recall} = \frac{TP}{TP + FN}
$$

- **TP** — истинно положительные (True Positives)
- **FP** — ложно положительные (False Positives)
- **FN** — ложно отрицательные (False Negatives)

---

### 2. **MCC (Matthews Correlation Coefficient)**

MCC — метрика, подходящая при сильной несбалансированности классов. Принимает значение от `-1` до `1`:

$$
\text{MCC} = \frac{TP \cdot TN - FP \cdot FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
$$

- `1` — идеально
- `0` — случайное предсказание
- `-1` — полное несоответствие

---

### 3. **Accuracy (доля верно классифицированных примеров)**

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

Используется, когда классы относительно сбалансированы.

---

## 🛠️ Технические детали

### 📦 Данные

Данные загружаются из HuggingFace:

```python
from datasets import load_dataset

dataset = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks", "promoter_all")
```

Каждая задача включает поля:
- `"sequence"` — строка из символов {A, T, G, C}
- `"label"` — таргет

---

### 🧪 Базовая схема эксперимента

1. 📥 Загрузить данные
2. 🧬 Преобразовать последовательности в векторы (эмбеддинги).
3. 🤖 Обучить классификатор.
4. 📊 Посчитать метрику на test.

---

## 🏆 Задание

Постройте pipeline, который даёт **максимально высокие значения метрик на всех 5 задачах**, применяя разные подходы к построению эмбеддингов ДНК-цепочек и решению предложенных задач классификации. Итоговое сравнение решений будет происходить по среднему значению всех 5 метрик.

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, f1_score, accuracy_score
from catboost import CatBoostClassifier
from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers.models.bert.configuration_bert import BertConfig
from tqdm import tqdm

import numpy as np

In [2]:
# Проверяем доступность CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Определение собственного класса Dataset для работы с последовательностями ДНК
class NTDataset(Dataset):
    def __init__(self, sequences, labels, k=6):
        self.sequences = sequences
        self.labels = labels
        self.k = k

    def __len__(self):
        return len(self.sequences)

    def _to_kmers(self, seq):
        return " ".join([seq[i:i+self.k] for i in range(len(seq) - self.k + 1)])

    # Если нужны k-меры
    # def __getitem__(self, idx):
    #     kmers = self._to_kmers(self.sequences[idx])
    #     return kmers, self.labels[idx]

    # Если нужны не k-меры, а исходная цепочка
    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        return sequence, self.labels[idx]

In [4]:
def load_nucleotide_transformer(batch_size, valid_split=-1, dataset_name='enhancers', split_state=42):
    # Загружаем тренировочный и тестовый наборы данных
    train_dataset = load_dataset(
        "InstaDeepAI/nucleotide_transformer_downstream_tasks",
        dataset_name,
        split="train",
        streaming=False,
    )

    test_dataset = load_dataset(
        "InstaDeepAI/nucleotide_transformer_downstream_tasks",
        dataset_name,
        split="test",
        streaming=False,
    )

    # Извлекаем последовательности и метки
    train_sequences = train_dataset['sequence']
    train_labels = train_dataset['label']

    # Если указано разделение на валидационный набор, разбиваем тренировочные данные
    if valid_split > 0:
        train_sequences, validation_sequences, train_labels, validation_labels = train_test_split(
            train_sequences, train_labels, test_size=valid_split, random_state=split_state
        )

    test_sequences = test_dataset['sequence']
    test_labels = test_dataset['label']

    # Создаем объекты класса NTDataset
    train_dataset = NTDataset(train_sequences, train_labels)
    test_dataset = NTDataset(test_sequences, test_labels)

    # Создаем DataLoader'ы для обучения и тестирования
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    # Создаем DataLoader для валидации, если есть разделение
    if valid_split > 0:
        validation_dataset = NTDataset(validation_sequences, validation_labels)
        valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
        
        print(f"Train: {len(train_loader.dataset)}, Validation: {len(valid_loader.dataset)}, Test: {len(test_loader.dataset)}")
        return train_loader, valid_loader, test_loader

    # Если нет разделения на валидацию, выводим количество образцов
    print(f"Train: {len(train_loader.dataset)}, Test: {len(test_loader.dataset)}")
    return train_loader, None, test_loader

In [5]:
# Получаем эмбеддинги DNA-BERT-2
def get_dnabert2_embeddings(dataloader, tokenizer, model, pooling="mean"):
    embeddings = []
    labels = []
    model.to(device)

    for batch in tqdm(dataloader):
        sequences, lbls = batch  # уже чистые строки

        tokens = tokenizer(
            list(sequences),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**tokens)
            hidden_states = outputs[0]  # shape: [batch_size, seq_len, hidden_dim]

        if pooling == "mean":
            pooled = hidden_states.mean(dim=1)  # [batch_size, hidden_dim]
        elif pooling == "max":
            pooled = hidden_states.max(dim=1)[0]  # [batch_size, hidden_dim]
        else:
            raise ValueError("pooling must be 'mean' or 'max'")

        embeddings.append(pooled.cpu().numpy())
        labels.append(lbls.numpy())

    return np.vstack(embeddings), np.concatenate(labels)

In [ ]:
# Обучаем классификатор на эмбеддингах DNA-BERT
def classify_with_dnabert(dataset_name, metric, embedding_fn = get_dnabert2_embeddings):
    # Поскольку в CatBoost нет Early Stopping по метрике MCC, используется в данном случае F1
    eval_metric = "F1" if metric == "MCC" else metric
        
    batch_size = 128 # Размер батча можно сделать поменьше при необходимости
    train_loader, valid_loader, test_loader = load_nucleotide_transformer(batch_size, valid_split=0.1, dataset_name=dataset_name)

    # DNA-BERT2: 117M параметров
    model_name = "zhihan1996/DNABERT-2-117M"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    config = BertConfig.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name, config=config, trust_remote_code=True)
    model.eval()

    X_train, y_train = embedding_fn(train_loader, tokenizer, model)
    X_valid, y_valid = embedding_fn(valid_loader, tokenizer, model)
    X_test, y_test = embedding_fn(test_loader, tokenizer, model)

    clf = CatBoostClassifier(
        iterations=3_000,
        learning_rate=0.02,
        depth=4,
        task_type="GPU",
        eval_metric=eval_metric,
        early_stopping_rounds=100,
        use_best_model=True,
        verbose=50
    )

    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid) if valid_loader else None)

    y_pred = clf.predict(X_test)

    if metric == "F1":
        metric = f1_score(y_test, y_pred, average="binary")
        print(f"F1 Score: {metric:.4f}")
    elif metric == "Accuracy":
        metric = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {metric:.4f}")
    else:
        metric = matthews_corrcoef(y_test, y_pred)
        print(f"MCC: {metric:.4f}")

    return metric

In [6]:
# Список датасетов, на которых будет измеряться метрика во время DNAChallenge
# В оригинальном бенчмарке 18 датасетов, для простоты решения задачи ограничимся пятью

# Если при воспроизведении пайплайна возникают ошибки, связанные с DNA-BERT-2, возможно, полезной окажется ссылка
# https://huggingface.co/mosaicml/mpt-7b-storywriter/discussions/10

DATASETS = [
    ("promoter_all", "F1"),
    ("enhancers", "MCC"),
    ("splice_sites_all", "Accuracy"),
    ("H3", "MCC"),
    ("H4", "MCC")
]

result_metrics = {}

for dataset, metric in DATASETS:
    result_metrics[dataset] = classify_with_dnabert(dataset, metric)

Train: 47948, Validation: 5328, Test: 5920


/home/ledneva/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████| 47/47 [00:15<00:00,  3.10it/s]


0:	learn: 0.8042558	test: 0.8047600	best: 0.8047600 (0)	total: 57.3ms	remaining: 2m 51s
50:	learn: 0.8738135	test: 0.8755162	best: 0.8755162 (50)	total: 1.59s	remaining: 1m 32s
100:	learn: 0.8960472	test: 0.8952530	best: 0.8954697 (99)	total: 3.09s	remaining: 1m 28s
150:	learn: 0.9067555	test: 0.9069949	best: 0.9069949 (148)	total: 4.59s	remaining: 1m 26s
200:	learn: 0.9144889	test: 0.9131620	best: 0.9139073 (198)	total: 6.08s	remaining: 1m 24s
250:	learn: 0.9198816	test: 0.9177203	best: 0.9181730 (245)	total: 7.58s	remaining: 1m 23s
300:	learn: 0.9228594	test: 0.9205362	best: 0.9205362 (300)	total: 9.1s	remaining: 1m 21s
350:	learn: 0.9254229	test: 0.9226894	best: 0.9236138 (340)	total: 10.6s	remaining: 1m 20s
400:	learn: 0.9272378	test: 0.9261693	best: 0.9265559 (396)	total: 12.1s	remaining: 1m 18s
450:	learn: 0.9295516	test: 0.9270270	best: 0.9272060 (447)	total: 13.7s	remaining: 1m 17s
500:	learn: 0.9314233	test: 0.9275922	best: 0.9285162 (488)	total: 15.2s	remaining: 1m 15s
550:	l

/home/ledneva/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|███████████████████████████████████████████| 4/4 [00:00<00:00,  6.09it/s]


0:	learn: 0.7689612	test: 0.7807768	best: 0.7807768 (0)	total: 27.4ms	remaining: 1m 22s
50:	learn: 0.8065608	test: 0.8277669	best: 0.8283093 (43)	total: 1.29s	remaining: 1m 14s
100:	learn: 0.8206790	test: 0.8389218	best: 0.8409987 (98)	total: 2.56s	remaining: 1m 13s
150:	learn: 0.8338983	test: 0.8509521	best: 0.8517060 (148)	total: 3.83s	remaining: 1m 12s
200:	learn: 0.8420742	test: 0.8550629	best: 0.8558201 (194)	total: 5.1s	remaining: 1m 11s
250:	learn: 0.8497822	test: 0.8569536	best: 0.8584656 (224)	total: 6.38s	remaining: 1m 9s
300:	learn: 0.8548637	test: 0.8687003	best: 0.8687003 (300)	total: 7.65s	remaining: 1m 8s
350:	learn: 0.8610270	test: 0.8683511	best: 0.8692767 (320)	total: 8.9s	remaining: 1m 7s
400:	learn: 0.8658573	test: 0.8734261	best: 0.8740053 (399)	total: 10.2s	remaining: 1m 5s
450:	learn: 0.8706405	test: 0.8740053	best: 0.8759124 (413)	total: 11.4s	remaining: 1m 4s
500:	learn: 0.8746386	test: 0.8770764	best: 0.8785667 (482)	total: 12.7s	remaining: 1m 3s
550:	learn: 0

...

0:	learn: 0.4313169	test: 0.4229630	best: 0.4229630 (0)	total: 24ms	remaining: 1m 12s
50:	learn: 0.4623045	test: 0.4574074	best: 0.4611111 (16)	total: 360ms	remaining: 20.8s
100:	learn: 0.4711111	test: 0.4666667	best: 0.4700000 (64)	total: 716ms	remaining: 20.5s
150:	learn: 0.4827984	test: 0.4685185	best: 0.4707407 (130)	total: 1.08s	remaining: 20.3s
200:	learn: 0.4917695	test: 0.4677778	best: 0.4714815 (173)	total: 1.46s	remaining: 20.3s
250:	learn: 0.5012346	test: 0.4625926	best: 0.4714815 (173)	total: 1.83s	remaining: 20s
bestTest = 0.4714814815
bestIteration = 173
Shrink model to first 174 iterations.
Accuracy: 0.4607
Train: 12121, Validation: 1347, Test: 1497


/home/ledneva/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████| 12/12 [00:05<00:00,  2.02it/s]


0:	learn: 0.7980851	test: 0.8034682	best: 0.8034682 (0)	total: 23.7ms	remaining: 1m 11s
50:	learn: 0.8184751	test: 0.8292683	best: 0.8292683 (48)	total: 1.15s	remaining: 1m 6s
100:	learn: 0.8259879	test: 0.8359088	best: 0.8376194 (97)	total: 2.25s	remaining: 1m 4s
150:	learn: 0.8338464	test: 0.8416422	best: 0.8424908 (134)	total: 3.36s	remaining: 1m 3s
200:	learn: 0.8394514	test: 0.8421829	best: 0.8434974 (168)	total: 4.47s	remaining: 1m 2s
250:	learn: 0.8435688	test: 0.8428044	best: 0.8445099 (232)	total: 5.59s	remaining: 1m 1s
300:	learn: 0.8472421	test: 0.8491124	best: 0.8491124 (284)	total: 6.7s	remaining: 1m
350:	learn: 0.8505564	test: 0.8507795	best: 0.8507795 (350)	total: 7.81s	remaining: 59s
400:	learn: 0.8551625	test: 0.8507795	best: 0.8514116 (380)	total: 8.92s	remaining: 57.8s
450:	learn: 0.8574616	test: 0.8507795	best: 0.8516320 (430)	total: 10s	remaining: 56.6s
500:	learn: 0.8601276	test: 0.8522643	best: 0.8537491 (473)	total: 11.1s	remaining: 55.4s
550:	learn: 0.8621762	t

/home/ledneva/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████| 12/12 [00:05<00:00,  2.07it/s]


0:	learn: 0.7529952	test: 0.7460757	best: 0.7460757 (0)	total: 25.3ms	remaining: 1m 16s
50:	learn: 0.8340607	test: 0.8251121	best: 0.8275246 (46)	total: 1.19s	remaining: 1m 8s
100:	learn: 0.8444101	test: 0.8377897	best: 0.8377897 (98)	total: 2.31s	remaining: 1m 6s
150:	learn: 0.8509370	test: 0.8413547	best: 0.8416370 (146)	total: 3.44s	remaining: 1m 4s
200:	learn: 0.8565668	test: 0.8449956	best: 0.8464951 (189)	total: 4.55s	remaining: 1m 3s
250:	learn: 0.8629218	test: 0.8447205	best: 0.8464951 (189)	total: 5.66s	remaining: 1m 2s
300:	learn: 0.8666029	test: 0.8467671	best: 0.8467671 (275)	total: 6.77s	remaining: 1m
350:	learn: 0.8702144	test: 0.8492908	best: 0.8492908 (326)	total: 7.88s	remaining: 59.5s
400:	learn: 0.8720008	test: 0.8510638	best: 0.8523431 (376)	total: 8.99s	remaining: 58.3s
450:	learn: 0.8742355	test: 0.8477876	best: 0.8523431 (376)	total: 10.1s	remaining: 57.1s
bestTest = 0.8523430592
bestIteration = 376
Shrink model to first 377 iterations.
MCC: 0.7287


In [8]:
# Максимальная ширина для выравнивания
max_name_len = max(len(name) for name in result_metrics.keys())
line_format = f"{{:<{max_name_len}}} : {{:.4f}}"

# Выводим таблицу
print("Metrics per dataset:\n")
for dataset, score in result_metrics.items():
    print(line_format.format(dataset, score))

# Среднее значение
mean_score = sum(result_metrics.values()) / len(result_metrics)
print("\n" + "-" * (max_name_len + 12))
print(f"{'Average'.ljust(max_name_len)} : {mean_score:.4f}")

Metrics per dataset:

promoter_all     : 0.9249
enhancers        : 0.4453
splice_sites_all : 0.4607
H3               : 0.6872
H4               : 0.7287

----------------------------
Average          : 0.6493
